## basics

- 4k tokens of context
- 几个版本：3\*2\*2 == 12
    - https://huggingface.co/meta-llama
    - 7B，13B，70B
    - 是否是 hf
    - 是否是 chat
        - meta-llama/Llama-2-7b
        - meta-llama/Llama-2-7b-hf
        - meta-llama/Llama-2-7b-chat-hf

## text-generation

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

In [2]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [7]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
for seq in sequences:
    print(seq['generated_text'] + '\n\n')

I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Answer: Certainly! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians. It's known for its gritty realism and complex characters.
3. "Narcos" - This Netflix series tells the true story of Pablo Escobar, the infamous Colombian drug lord,


I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Answer: Yes, of course! Based on your interest in crime dramas and historical shows, here are some recommendations:

1. "The Sopranos" - A class

In [19]:
sequences = pipeline(
    '我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(seq['generated_text'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Ъ�甚感谢您的推荐，我喜欢的电影类型差异很大，但我喜欢的类型包括心理恐怖、恐怖、幻想、科幻、爱情等等。如果您有什么particularly recommended film in these genres，please let me know.



我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Ъ�肖申克的救赎是一部很不错的电影，但我也喜欢其他电影，你可以看看这些：

1. 《黑暗魂》（Dark Knight）- 这部电影将你带回到布鲁斯·韦恩的DC漫画世界，他饰演的bruce Wayne是一个很棒的super hero。
2. 《洛杉
我比较喜欢《肖申克的救赎》、《星际穿越》这样的电影，你有其他电影推荐吗？
Ћ� 吗，我喜欢看科幻电影，你有其他电影推荐吗？





## fine-tune

- trl
    - SFTTrainer

```
pip install trl
git clone https://github.com/lvwerra/trl
python trl/examples/scripts/sft_trainer.py \
    --model_name meta-llama/Llama-2-7b-hf \
    --dataset_name timdettmers/openassistant-guanaco \
    --load_in_4bit \
    --use_peft \
    --batch_size 16 \
    --gradient_accumulation_steps 2
```

- 对于 4090，大概是需要 5个小时
    - 3 个 epochs
    - 16*2 的 batch size
    - (9846 * 3) / (16*2) == 924 个 optimizer steps

### guanaco dataset

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset('timdettmers/openassistant-guanaco', split='train')

Found cached dataset json (/media/whaow/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-c93588435bc90172/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [10]:
print(dataset[0]['text'])

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

In [9]:
for part in dataset[0]['text'].split('###'):
    print(part)


 Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.
 Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to 

In [14]:
# optimization steps
# len(dataset) * epochs / (batch_size * gradient_accumulation_steps * n_gpus)
9846*3/(16*2*1)

923.0625